# Diffusion Denoising 모델 학습


## 1. 환경 설정 및 라이브러리 설치


In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# 필요한 라이브러리 설치
%pip install diffusers accelerate transformers loguru --quiet


In [ ]:
# 프로젝트 경로 설정 및 라이브러리 경로 추가
import sys
import os

PROJECT_PATH = '/content/drive/MyDrive/Data Scientist/Project/Week5/week5'
sys.path.append(PROJECT_PATH)


## 2. 학습 설정 (Configuration)


In [ ]:
import torch
from dataclasses import dataclass
from pathlib import Path
import os

# 기존 프로젝트 설정 가져오기
from params import config as global_config

@dataclass
class TrainingConfig:
    image_size = global_config.image_size[0]  # 이미지 크기 (정사각형 가정)
    train_batch_size = 16
    eval_batch_size = 16
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 10
    mixed_precision = 'fp16'  # 'no', 'fp16', 'bf16'
    output_dir = os.path.join(PROJECT_PATH, 'logs_diffusion_denoiser')  # 모델 저장 경로

    # 데이터로더 설정
    train_data_dir = global_config.train_dataset[0]
    data_type = global_config.data_type
    noise_type = global_config.noise_type
    noise_levels = global_config.noise_levels

config = TrainingConfig()

# 출력 디렉토리 생성
if not os.path.exists(config.output_dir):
    os.makedirs(config.output_dir)


## 3. 데이터 준비


In [ ]:
from code_denoising.datawrapper.datawrapper import ControlledDataWrapper, DataKey
from torch.utils.data import DataLoader

# ControlledDataWrapper를 사용하여 학습 데이터셋 로드
# augmentation_mode를 'noise_only'로 설정하여 노이즈만 적용된 이미지와 원본 이미지를 받습니다.
dataset = ControlledDataWrapper(
    file_path=[config.train_data_dir],
    data_type=config.data_type,
    training_mode='denoising',  # 'denoising' 또는 'deconvolution' 등, 여기서는 노이즈 제거가 목표
    augmentation_mode='noise_only',
    noise_type=config.noise_type,
    noise_levels=config.noise_levels,
    conv_directions=[] # 노이즈만 학습하므로 conv는 비워둡니다.
)

train_dataloader = DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

print(f"Successfully loaded {len(dataset)} training images.")


## 4. Diffusion 모델 및 스케줄러 정의


In [ ]:
from diffusers import UNet2DModel, DDPMScheduler

# U-Net 모델 정의
# 우리 데이터는 1채널(grayscale) 이미지이므로 in_channels=1, out_channels=1로 설정합니다.
model = UNet2DModel(
    sample_size=config.image_size,
    in_channels=1,  # 흑백 이미지
    out_channels=1, # 노이즈 예측 결과도 흑백
    layers_per_block=2,
    block_out_channels=(128, 128, 256, 256, 512, 512),
    down_block_types=(
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",
        "AttnUpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D"
    ),
)

# 노이즈 스케줄러 정의
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

# 옵티마이저 정의
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)


## 5. 학습 루프 (Training Loop)


In [ ]:
import torch.nn.functional as F
from diffusers.optimization import get_cosine_schedule_with_warmup
from accelerate import Accelerator
from tqdm.auto import tqdm
from pathlib import Path
import numpy as np
from PIL import Image

# 학습률 스케줄러
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

# Accelerator 초기화
accelerator = Accelerator(
    mixed_precision=config.mixed_precision,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    log_with="tensorboard",
    project_dir=os.path.join(config.output_dir, "runs")
)

if accelerator.is_main_process:
    accelerator.init_trackers("train_diffusion_denoiser")

# 모델, 옵티마이저, 데이터로더 준비
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

global_step = 0
# 학습 시작
for epoch in range(config.num_epochs):
    progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
    progress_bar.set_description(f"Epoch {epoch}")

    for step, batch in enumerate(train_dataloader):
        clean_images = batch[DataKey.image_gt] # 1채널 원본 이미지
        
        # 1. 순수 노이즈 생성
        noise = torch.randn(clean_images.shape).to(clean_images.device)
        bs = clean_images.shape[0]

        # 2. 무작위 시간 단계(timestep) 샘플링
        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device).long()

        # 3. 노이즈 추가
        noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

        with accelerator.accumulate(model):
            # 4. 모델의 노이즈 예측
            noise_pred = model(noisy_images, timesteps, return_dict=False)[0]

            # 5. Loss 계산 (예측된 노이즈와 실제 노이즈 비교)
            loss = F.mse_loss(noise_pred, noise)
            accelerator.backward(loss)

            accelerator.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        progress_bar.update(1)
        logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
        progress_bar.set_postfix(**logs)
        accelerator.log(logs, step=global_step)
        global_step += 1

    # 에폭마다 모델 저장
    if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
        if accelerator.is_main_process:
            unwrapped_model = accelerator.unwrap_model(model)
            unwrapped_model.save_pretrained(os.path.join(config.output_dir, f"checkpoint_epoch_{epoch}"))
            print(f"Saved model checkpoint at epoch {epoch}")

print("Training finished.")
